In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
# Store csv created in part one into a DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,portsmouth,50.80,-1.09,55.99,87,12,4.63,GB,1595376734
1,1,tautira,-17.73,-149.15,84.20,66,40,5.82,PF,1595376550
2,2,wukari,7.85,9.78,73.63,91,87,8.68,NG,1595376734
3,3,namatanai,-3.67,152.43,84.33,70,27,6.13,PG,1595376734
4,4,saint-leu,-21.15,55.28,64.40,68,90,5.82,RE,1595376734


## Humidity Heatmap

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## Create new DataFrame fitting weather criteria

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,giffoni valle piana,40.72,14.94,75.99,59,0,3.04,IT,1595376738
71,71,soyo,-6.13,12.37,70.48,89,0,7.99,AO,1595376568
86,86,botwood,49.15,-55.35,71.01,63,0,7.58,CA,1595376747
120,120,meliki,40.52,22.40,73.00,59,0,5.10,GR,1595376752
137,137,yagodnoye,62.55,149.67,72.95,40,0,4.61,RU,1595376491
180,180,maceio,-9.67,-35.74,71.60,83,0,1.12,BR,1595376623
215,215,jawa,31.85,35.94,71.60,24,0,7.31,JO,1595376766
226,226,mahon,39.89,4.27,73.40,83,0,8.05,ES,1595376768
237,237,suez,29.97,32.53,73.65,81,0,8.70,EG,1595376769
287,287,baruun-urt,46.68,113.28,73.00,51,0,8.61,MN,1595376778


## Hotel Map

In [6]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
27,giffoni valle piana,IT,40.72,14.94,
71,soyo,AO,-6.13,12.37,
86,botwood,CA,49.15,-55.35,
120,meliki,GR,40.52,22.40,
137,yagodnoye,RU,62.55,149.67,
180,maceio,BR,-9.67,-35.74,
215,jawa,JO,31.85,35.94,
226,mahon,ES,39.89,4.27,
237,suez,EG,29.97,32.53,
287,baruun-urt,MN,46.68,113.28,


In [7]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
27,giffoni valle piana,IT,40.72,14.94,Albergo Le Dodici Lune Di Alessandra E Paola C...
71,soyo,AO,-6.13,12.37,Hotel Mpampa
86,botwood,CA,49.15,-55.35,Exploits Hostel
120,meliki,GR,40.52,22.40,Ampelótopos
137,yagodnoye,RU,62.55,149.67,Guest House Yagodnoe
180,maceio,BR,-9.67,-35.74,Mercure Maceio Pajucara Hotel
215,jawa,JO,31.85,35.94,دار الحاج عبدالله ابو صعيليك واولاده
226,mahon,ES,39.89,4.27,ARTIEM Capri
237,suez,EG,29.97,32.53,Green House
287,baruun-urt,MN,46.68,113.28,Ахмадын амралт сувилал


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))